<a href="https://colab.research.google.com/github/anirudh-cp/ISODS-NLP/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **The International Society of Data Scientists - NLP Track**
#### **Fashion and Beauty Review Rating - The 3rd Annual International Data Science & AI Competition 2022**


Predict how satisfied customers are based on their product reviews, to help with insight in fashion and beauty products.

#### Downloading Datasets

In [ ]:
%pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/competitions/fashion-and-beauty-reviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:Your Kaggle Key:

c:\Users\cpani\AppData\Local\Programs\Python\Python310\lib\site-packages\cryptography\hazmat\backends\openssl\x509.py:14: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  warnings.warn(


100%|██████████| 112M/112M [00:28<00:00, 4.15MB/s] 


Extracting archive .\fashion-and-beauty-reviews/fashion-and-beauty-reviews.zip to .\fashion-and-beauty-reviews


#### Working with the Dataset

In [ ]:
%pip install pandas
%pip install numpy
%pip install torch

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataframe = pd.read_csv(r"fashion-and-beauty-reviews\review_train.tsv", sep='\t', header=0)

In [ ]:
dataframe.head()

,rating,review,summary,product,reviewer
0,4.0,I received this cream about 8 days ago and hav...,Thought this cream was making me sick...,B00DKEQYJY,A3IQA3VVDHGAK1
1,5.0,Beautiful pieces,Five Stars,B00L4JJKH0,A1QWCVZMSYG1N2
2,5.0,Really impressive tree. It goes together quick...,Highly recommended,1620213982,A2QRLRHMFDJ25E
3,4.0,good,Four Stars,B01D2J35BG,A2MMO1P2ZNEUNF
4,5.0,Gift for granddaughter. She loved it.,Cool looking umbrella.,B01422IQD4,A1WXKDA47VPOFT


##### Simple Analysis

In [ ]:
dataframe.shape

(1003984, 5)

In [ ]:
print(f"Possible ratings : {np.sort(dataframe['rating'].unique())}")

Possible ratings : [1. 2. 3. 4. 5.]


In [ ]:
for col in ['review', 'summary']:
      print(f"Longest string in '{col}' is record# {dataframe[col].str.len().argmax()} "
            f"of length {len(dataframe.iloc[dataframe[col].str.len().argmax()][col])}")

      print(f"Shortest string in '{col}' is record# {dataframe[col].str.len().argmin()} "
            f"of length {len(dataframe.iloc[dataframe[col].str.len().argmin()][col])}")

Longest string in 'review' is record# 788873 of length 13741
Shortest string in 'review' is record# 9398 of length 1
Longest string in 'summary' is record# 800077 of length 267
Shortest string in 'summary' is record# 2321 of length 1


In [ ]:
dataframe.isna().sum()

rating         0
review      1304
summary      597
product        0
reviewer       0
dtype: int64

#### Set up

In [ ]:
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn

In [ ]:
EPOCHS = 10
BATCH_SIZE = 64
LEARN_RT = 0.001
PATIENCE = 5

OPTIMIZER = torch.optim.Adam
SCHEDULER = torch.optim.lr_scheduler.ReduceLROnPlateau

In [ ]:
BASE_PATH = "/content/drive/MyDrive/ISODS/"

In [ ]:
def set_seeds(seed=1234):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # multi-GPU

In [ ]:
import time

set_seeds(seed=int(time.time()))

In [ ]:
# Set device
cuda = True
device = torch.device("cuda" if (
    torch.cuda.is_available() and cuda) else "cpu")
torch.set_default_tensor_type("torch.FloatTensor")
if device.type == "cuda":
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
print (device)

cpu


#### Data Pre-processing

In [ ]:
dataframe = pd.read_csv(r"fashion-and-beauty-reviews\review_train.tsv", sep='\t', header=0)

In [ ]:
# Randomize/Shuffle order of all records in the dataset
dataframe = dataframe.sample(frac=1).reset_index(drop=True)

In [ ]:
dataframe.head()

,rating,review,summary,product,reviewer
0,5.0,fabric is easy care. am 5'3. hits me about 2...,great flip skirt for ballroom dancing,B00GN72LYC,A26ZA7I89DN029
1,5.0,NaN,Five Stars,B00UAWE978,A3P3UITN9EQIW4
2,5.0,Got this for my mom. She loved it. It is reall...,She loved it. It is really cute,B00OB8WRBW,A226AVPBKYZUU0
3,5.0,Great product. Great seller,Five Stars,B00FGCU5M0,A2JL10RY5CI9AM
4,5.0,I love this necklace. It's so pretty and looks...,I love this necklace,B01ADTLECK,A2TPTTT3JVFJL6


In [ ]:
# Remove product and reviewer information and combine review and summary into one column
dataframe.drop(['product', 'reviewer'], axis=1, inplace=True)
dataframe['text'] = dataframe['summary'] + ' ' +dataframe['review']
dataframe.drop(['review', 'summary'], axis=1, inplace=True)

dataframe.dropna(inplace=True)
dataframe.isna().sum()

rating    0
text      0
dtype: int64

In [ ]:
dataframe.head()

,rating,text
0,5.0,great flip skirt for ballroom dancing fabric i...
2,5.0,She loved it. It is really cute Got this for m...
3,5.0,Five Stars Great product. Great seller
4,5.0,I love this necklace I love this necklace. It'...
5,5.0,"Worthy the money Nice jeans, true to size. I w..."


In [ ]:
%pip install contractions
%pip install nltk
%pip install inflect

In [ ]:
import re
from string import punctuation
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
import inflect

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = set(stopwords.words('english')) - {'against', 'few', 'more', 'no', 'nor', 'not', 'very'}
lemma = WordNetLemmatizer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cpani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cpani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cpani\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\cpani\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
def process(text):

   text = re.sub(r'http\S+', '', text)             # Remove links
   text = re.sub(r"\([^)]*\)", "", text)           # Remove word within brackets
   text = re.sub(f'[{punctuation}]', '', text)     # Remove punctuation
      
   # If number and word are combined seperate with space
   text = re.sub(r'(\d*)(\D+)(\d*)', r'\1 \2 \3', text)
   
   text = re.sub(" +", " ", text)                  # Remove multiple spacing
   text = text.strip()
   
   text = text.lower()                             # Convert to lowercase
   text = contractions.fix(text)                   # Contraction replacement
   
   tokens = nltk.word_tokenize(text)               # Tokenize
   
   # Convert all numbeers to words
   for index, token in enumerate(tokens):
      if token.isdigit():
         try:
            tokens[index] = inflect.engine().number_to_words(token)
         except Exception:
            print(token)
            
   # Remove stopwords
   tokens = [token for token in tokens if token not in stop_words]
   
   # Lemmatization
   tokens = [lemma.lemmatize(token, pos='v') for token in tokens]
   
   text = ' '.join(tokens)
   
   return text

In [ ]:
dataframe['text'] = dataframe['text'].apply(process)

7791793621055041073741832752334614789979
4666286801020021073741825353887201376151820146361416897


In [ ]:
dataframe.to_csv(r"fashion-and-beauty-reviews\review_train_processed.tsv", sep='\t')

#### Label Encoding

In [ ]:
dataframe.dtypes

rating    float64
text       object
dtype: object

The point of label encoding is to convert any labels assigned to the output classes to an integer value. However, since this dataset has the output values as numbers already we can convert them to int from float.

In [ ]:
dataframe['rating'] = dataframe['rating'].astype(np.int64)

In [ ]:
dataframe['rating'] = dataframe['rating'] - 1

In [ ]:
print(f"Possible ratings : {np.sort(dataframe['rating'].unique())}")

In [ ]:
dataframe.dtypes

rating     int64
text      object
dtype: object

#### Data Split

In [ ]:
dataframe = pd.read_csv(BASE_PATH + r"fashion-and-beauty-reviews/review_train_processed.tsv", sep='\t', header=0, index_col=0)

In [ ]:
dataframe = pd.read_csv(r"fashion-and-beauty-reviews\review_train_processed.tsv", sep='\t', header=0, index_col=0)

In [ ]:
dataframe.dropna(inplace=True)
dataframe.isna().sum()

rating    0
text      0
dtype: int64

In [ ]:
%pip install sklearn

In [ ]:
import collections
from sklearn.model_selection import train_test_split

In [ ]:
# Configure for 80% data to be for training and the rest for validation here.

TRAIN_SIZE = 0.8
VAL_SIZE = 0.2

In [ ]:
# Data
X = dataframe['text'].values
y = dataframe['rating'].values

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=TRAIN_SIZE, stratify=y)

In [ ]:
print (f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print (f"X_val: {X_val.shape}, y_val: {y_val.shape}")

X_train: (801651,), y_train: (801651,)
X_val: (200413,), y_val: (200413,)


#### Tokenization

In [ ]:
import json
from collections import Counter

In [ ]:
class Tokenizer(object):
    def __init__(self, char_level, num_tokens=None,
                 pad_token="<PAD>", oov_token="<UNK>",
                 token_to_index=None):
        self.char_level = char_level
        self.separator = "" if self.char_level else " "
        if num_tokens: num_tokens -= 2 # pad + unk tokens
        self.num_tokens = num_tokens
        self.pad_token = pad_token
        self.oov_token = oov_token
        if not token_to_index:
            token_to_index = {pad_token: 0, oov_token: 1}
        self.token_to_index = token_to_index
        self.index_to_token = {v: k for k, v in self.token_to_index.items()}

    def __len__(self):
        return len(self.token_to_index)

    def __str__(self):
        return f"<Tokenizer(num_tokens={len(self)})>"

    def fit_on_texts(self, texts):
        if not self.char_level:
            texts = [text.split(" ") for text in texts]
        all_tokens = [token for text in texts for token in text]
        counts = Counter(all_tokens).most_common(self.num_tokens)
        self.min_token_freq = counts[-1][1]
        for token, count in counts:
            index = len(self)
            self.token_to_index[token] = index
            self.index_to_token[index] = token
        return self

    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            if not self.char_level:
                text = text.split(" ")
            sequence = []
            for token in text:
                sequence.append(self.token_to_index.get(
                    token, self.token_to_index[self.oov_token]))
            sequences.append(np.asarray(sequence))
        return sequences

    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = []
            for index in sequence:
                text.append(self.index_to_token.get(index, self.oov_token))
            texts.append(self.separator.join([token for token in text]))
        return texts

    def save(self, fp):
        with open(fp, "w") as fp:
            contents = {
                "char_level": self.char_level,
                "oov_token": self.oov_token,
                "token_to_index": self.token_to_index
            }
            json.dump(contents, fp, indent=4, sort_keys=False)

    @classmethod
    def load(cls, fp):
        with open(fp, "r") as fp:
            kwargs = json.load(fp=fp)
        return cls(**kwargs)


In [ ]:
# Tokenize
tokenizer = Tokenizer(char_level=False, num_tokens=500)
tokenizer.fit_on_texts(texts=X_train)
VOCAB_SIZE = len(tokenizer)
print (tokenizer)

<Tokenizer(num_tokens=500)>


In [ ]:
# Convert texts to sequences of indices
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

In [ ]:
# Sample of tokens
for index in range(5):
    print(tokenizer.index_to_token[index])
print (f"least freq token's freq: {tokenizer.min_token_freq}") # use this to adjust num_tokens

<PAD>
<UNK>
not
star
love
least freq token's freq: 4955


#### Dataloader

The dataloader is responsible for the loading of data in the form of batches to the CNN model. Since the one-hot encoding cannot be performed over the entirity of the dataset, we will perform one-hot encoding and padding over a batch instead.

##### One-hot encoding

In [ ]:
def to_categorical(seq, num_classes):
    """One-hot encode a sequence of tokens."""
    one_hot = np.zeros((len(seq), num_classes))
    for i, item in enumerate(seq):
        one_hot[i, item] = 1.
    return one_hot

##### Padding

In [ ]:
def pad_sequences(sequences, max_seq_len=0):
    """Pad sequences to max length in sequence."""
    max_seq_len = max(max_seq_len, max(len(sequence) for sequence in sequences))
    num_classes = sequences[0].shape[-1]
    padded_sequences = np.zeros((len(sequences), max_seq_len, num_classes))
    for i, sequence in enumerate(sequences):
        padded_sequences[i][:len(sequence)] = sequence
    return padded_sequences

##### Dataloader

In [ ]:
FILTER_SIZE = 1 # unigram

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, X, y, max_filter_size):
        self.X = X
        self.y = y
        self.max_filter_size = max_filter_size

    def __len__(self):
        return len(self.y)

    def __str__(self):
        return f"<Dataset(N={len(self)})>"

    def __getitem__(self, index):
        X = self.X[index]
        y = self.y[index]
        return [X, y]

    def collate_fn(self, batch):
        """Processing on a batch."""
        # Get inputs
        batch = np.array(batch)
        X = batch[:, 0]
        y = batch[:, 1]

        # Perform one-hot encoding
        X = [to_categorical(seq, num_classes=VOCAB_SIZE) for seq in X]

        # Pad sequences
        X = pad_sequences(X, max_seq_len=self.max_filter_size)

        # Cast
        X = torch.FloatTensor(X.astype(np.int32))
        y = torch.LongTensor(y.astype(np.int32))

        return X, y

    def create_dataloader(self, batch_size, shuffle=False, drop_last=False):
        return torch.utils.data.DataLoader(
            dataset=self, batch_size=batch_size, collate_fn=self.collate_fn,
            shuffle=shuffle, drop_last=drop_last, pin_memory=True)


In [ ]:
train_dataset = Dataset(X=X_train, y=y_train, max_filter_size=FILTER_SIZE)
val_dataset = Dataset(X=X_val, y=y_val, max_filter_size=FILTER_SIZE)

In [ ]:
train_dataloader = train_dataset.create_dataloader(batch_size=BATCH_SIZE)
val_dataloader = val_dataset.create_dataloader(batch_size=BATCH_SIZE)

#### Model

In [ ]:
class MLBase(nn.Module):
    
    def accuracy(self, outputs, labels):
        _, preds = torch.max(outputs, dim=1)
        return torch.tensor(torch.sum(preds == labels).item() / len(preds))

    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = self.accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    

    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))


In [ ]:
import torch.nn.functional as F
import math

NUM_FILTERS = OUT_CHANNELS = 50
STRIDE = 1
FILTER_SIZE = 1
HIDDEN_DIM = 100
DROPOUT_P = 0.1
NUM_CLASSES = 5

In [ ]:
class CNN(MLBase):
    def __init__(self):
        super().__init__()
        
        self.filter_size = FILTER_SIZE
        self.conv = nn.Conv1d(
            in_channels=VOCAB_SIZE, out_channels=OUT_CHANNELS,
            kernel_size=FILTER_SIZE, stride=1, padding=0, padding_mode="zeros")
        
        self.bn1 = nn.BatchNorm1d(num_features=OUT_CHANNELS)

        # FC layers
        self.fc1 = nn.Linear(NUM_FILTERS, HIDDEN_DIM)
        self.dropout = nn.Dropout(DROPOUT_P)
        self.fc2 = nn.Linear(HIDDEN_DIM, NUM_CLASSES)
        
    def forward(self, xb):

        x_in = xb.transpose(1, 2)
        max_seq_len = x_in.shape[2]
        padding_left = int((self.conv.stride[0]*(max_seq_len-1) - max_seq_len + self.filter_size)/2)
        padding_right = int(math.ceil((self.conv.stride[0]*(max_seq_len-1) - max_seq_len + self.filter_size)/2))

        # Conv outputs
        z = self.conv(F.pad(x_in, (padding_left, padding_right)))
        z = F.relu(self.bn1(z))
        z = F.max_pool1d(z, z.size(2)).squeeze(2)

        # FC layer
        z = self.fc1(z)
        z = self.dropout(z)
        z = self.fc2(z)
        return z

In [ ]:
model = CNN()

##### Optimizer and Scheduler

In [ ]:
optimizer = OPTIMIZER(model.parameters(), lr=LEARN_RT)
scheduler = SCHEDULER(optimizer, mode="min", factor=0.1, patience=3)

#### Training

In [ ]:
from datetime import datetime

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)


In [ ]:
def fit(model, train_loader, val_loader, optimizer, scheduler, patienceInit):
    history = []
    best_val_loss = np.inf
    patience = patienceInit
    
    for epoch in range(EPOCHS):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            optimizer.zero_grad()
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            
        # Validation phase
        result = evaluate(model, val_loader)
        
        # Change learning rate if required
        scheduler.step(result['val_loss'])
        
        # Early stopping
        if result['val_loss'] < best_val_loss:
            best_val_loss = result['val_loss']
            best_model = model
            patience = patienceInit  # reset _patience
        else:
            patience -= 1
        
        # Save model after each epoch
        torch.save(model.state_dict(), BASE_PATH + f'Models/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}_{result["val_acc"]}_{epoch}.pth')

        # Logging
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['epoch'] = epoch
        result['lr'] = optimizer.param_groups[0]['lr']
        result['patience'] = patience
        
        # model.epoch_end(epoch, result)
        history.append(result)
        
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {result['train_loss']:.5f}, "
            f"val_loss: {result['val_loss']:.5f}, "
            f"val_acc: { result['val_acc']:.5f}, "
            f"lr: {optimizer.param_groups[0]['lr']:.2E}, "
            f"patience: {patience}"
        )
        
        if not patience:  # 0
            print("Stopping early!")
            break
    
    return history


In [ ]:
history = fit(model, train_dataloader, val_dataloader, optimizer, scheduler, PATIENCE)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


IndexError: ignored

#### Evaluation

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs')
    plt.savefig(BASE_PATH + f'Analysis/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}_Accuracy_vs_Epoch.png')
    # plt.show()

In [ ]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.savefig(BASE_PATH + f'Analysis/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}_Loss_vs_Epoch.png')
    # plt.show()

In [ ]:
plot_accuracies(history)
plot_losses(history)

In [ ]:
with open(f'Analysis/{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}_History.json', 'w', encoding ='utf8') as json_file:
  json.dump(history, json_file, ensure_ascii = True, indent=4)
